# Interface for Pipeline Execution
## (EUR-USD exchange data)

In [1]:
# Import models, metrics, paths and functions
from utils.helpers import csv_reader
from utils.paths import *
from pipeline.run_pipeline import run_pipeline

from models import MODELS
from metrics import METRICS

Loading paths...
Loading helper functions...
Loading data transformers...
Loading models...
Loading metrics...


In [4]:
# Get current models and metrics
print(METRICS)
print(MODELS)

{'MAE': <function mean_absolute_error at 0x000002D98E3F5620>, 'RMSE': <function root_mean_squared_error at 0x000002D98E520220>, 'MAPE': <function mean_absolute_percentage_error at 0x000002D98E3F59E0>, 'sMAPE': <function symmetric_mean_absolute_percentage_error at 0x000002D98E520360>}
{'FORECASTERS': {'without_covariates': {'Naive': {'model': <class 'sktime.forecasting.naive.NaiveForecaster'>, 'package': 'sktime', 'options': {'strategy': 'last'}}, 'Naive (drift)': {'model': <class 'sktime.forecasting.naive.NaiveForecaster'>, 'package': 'sktime', 'options': {'strategy': 'drift'}}, 'AutoSARIMA': {'model': <class 'sktime.forecasting.arima.AutoARIMA'>, 'package': 'sktime', 'options': {'seasonal': True, 'stationary': False, 'd': 1, 'trace': False, 'update_pdq': False, 'with_intercept': 'auto', 'max_p': 6, 'max_q': 6, 'suppress_warnings': True}}, 'Exponential Smoothing': {'model': <class 'sktime.forecasting.exp_smoothing.ExponentialSmoothing'>, 'package': 'sktime', 'options': {}}, 'STL': {'mo

In [9]:
## Using the EUR-USD Exchange Rate data

# Read EUR-USD exchange data
df = csv_reader(TESTDATA_DIR, 'eurusd', columns=['datetime', 'bid_close'])

# Use a small subset of the original data for testing:
df = df.iloc[:10000, :]
display(df)

,bid_close
datetime,
2020-01-01 00:00:00,NaN
2020-01-01 00:15:00,NaN
2020-01-01 00:30:00,NaN
2020-01-01 00:45:00,NaN
2020-01-01 01:00:00,NaN
...,...
2020-04-14 02:45:00,1.09466
2020-04-14 03:00:00,1.09463
2020-04-14 03:15:00,1.09456


In [10]:
output_dict = (
    run_pipeline(
        df=df, models=MODELS, metrics=METRICS,
        agg_method='last', agg_freq='B',
        forecast_init_train=0.3, ensemble_init_train=0.3,
        autosarimax_refit_interval=0.33,
        select_forecasters=['Naive', 'STL'],
        select_ensemblers=['Simple', 'Inverse RMSE'],
        verbose=True
    )
)

[2024-03-06 18:29] Starting  Pipeline...

== Starting Step 1 in Pipeline: Data Preprocessing ==

Searching for time information...
Dates found in 'index' column!
Inferred frequency: 15T
Data from goes from 2020-01-01 to 2020-04-14, resulting in 10000 observations.

Aggregating data to frequency 'business day' using method 'last' and dropping NaNs...
...finished!
Data now has 74 observations.

Selecting target and covariates...
Target: bid_close
Covariates: None

Data Insight:

            bid_close
datetime             
2020-01-02    1.11702
2020-01-03    1.11599
2020-01-06    1.11948
2020-01-07    1.11530
2020-01-08    1.11124
[Time elapsed: 00s]


== Starting Step 2 in Pipeline: Individual Forecasts ==

Splitting data (train/test ratio: 30/70)...
Initial training set has 22 observations 
and goes from 2020-01-02 to 2020-01-31
There are 52 periods to be forecasted: 
2020-02-03 to 2020-04-14

Now generating 52 one-step ahead expanding window predictions from model: Naive (sktime)
...fi

In [13]:
import pandas as pd

pd.set_option('display.float_format', '{:.2f}'.format)
# Format performance metric values
def format_numeric(val):
    if isinstance(val, float):
        return '{:.3f}'.format(val)
    return val
formatted_metrics = output_dict['metrics ranking'].applymap(format_numeric)

display(formatted_metrics.style.hide())


Model,MAE,RMSE,MAPE,sMAPE,MAE Ranking,RMSE Ranking,MAPE Ranking,sMAPE Ranking
Naive,0.007,0.009,0.006,0.403,1,1,1,1
Weighted Ensemble: Inverse RMSE,0.009,0.011,0.008,0.545,2,2,2,2
Weighted Ensemble: Simple,0.010,0.012,0.009,0.604,3,3,3,3
STL,0.014,0.018,0.013,0.860,4,4,4,4
